In [ ]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.3 MB/s eta 0:00:00


In [2]:
import torch
print(torch.__version__)

2.3.0+cu121


In [3]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.2.1+cpu.html

Looking in links: https://data.pyg.org/whl/torch-2.2.1+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.1/508.1 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.0/770.0 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.9/213.9 kB 24.5 MB/s eta 0:00:00


#4.1

In [4]:
from torch.optim import Adam
from torch_geometric.datasets import FB15k_237
from torch_geometric.nn import TransE, ComplEx
import torch
from torch_geometric.nn import TransE, ComplEx

#Exercise 1 – Creating network nesting for a knowledge graph with TransE

In [ ]:
def train(model, data_loader, optimizer, epochs=5):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        total_examples = 0

        for head_index, rel_type, tail_index in data_loader:
            optimizer.zero_grad()
            loss = model.loss(head_index, rel_type, tail_index)
            loss.backward()
            optimizer.step()
            total_loss += float(loss) * head_index.numel()
            total_examples += head_index.numel()

        loss = total_loss / total_examples
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

In [ ]:
def evaluate(model, data_loader):
    hits1_list = []
    hits3_list = []
    hits10_list = []
    mr_list = []
    mrr_list = []

    for head_index, rel_type, tail_index in data_loader:
        head_embeds = model.node_emb(head_index)
        relation_embeds = model.rel_emb(rel_type)
        tail_embeds = model.node_emb(tail_index)

        if isinstance(model, TransE):
            scores = torch.norm(head_embeds + relation_embeds - tail_embeds, p=1, dim=1)

        elif isinstance(model, ComplEx):
            # Get real and imaginary parts
            re_relation, im_relation = torch.chunk(relation_embeds, 2, dim=1)
            re_head, im_head = torch.chunk(head_embeds, 2, dim=1)
            re_tail, im_tail = torch.chunk(tail_embeds, 2, dim=1)

            # Compute scores
            re_score = re_head * re_relation - im_head * im_relation
            im_score = re_head * im_relation + im_head * re_relation
            scores = (re_score * re_tail + im_score * im_tail)

            # Negate as we want to rank scores in ascending order, lower the better
            scores = - scores.sum(dim=1)

        else:
            raise ValueError(f'Unsupported model.')

        scores = scores.view(-1, head_embeds.size()[0])

        hits1, hits3, hits10, mr, mrr = eval_metrics(scores)
        hits1_list.append(hits1.item())
        hits3_list.append(hits3.item())
        hits10_list.append(hits10.item())
        mr_list.append(mr.item())
        mrr_list.append(mrr.item())

    hits1 = sum(hits1_list) / len(hits1_list)
    hits3 = sum(hits3_list) / len(hits1_list)
    hits10 = sum(hits10_list) / len(hits1_list)
    mr = sum(mr_list) / len(hits1_list)
    mrr = sum(mrr_list) / len(hits1_list)

    return hits1, hits3, hits10, mr, mrr

In [ ]:
def eval_metrics(y_pred):
    argsort = torch.argsort(y_pred, dim=1, descending=False)
    # not using argsort to do the rankings to avoid bias when the scores are equal
    ranking_list = torch.nonzero(argsort == 0, as_tuple=False)
    ranking_list = ranking_list[:, 1] + 1
    hits1_list = (ranking_list <= 1).to(torch.float)
    hits3_list = (ranking_list <= 3).to(torch.float)
    hits10_list = (ranking_list <= 10).to(torch.float)
    mr_list = ranking_list.to(torch.float)
    mrr_list = 1. / ranking_list.to(torch.float)

    return hits1_list.mean(), hits3_list.mean(), hits10_list.mean(), mr_list.mean(), mrr_list.mean()

In [ ]:
train_data = FB15k_237('../data/FB15k', split='train')[0]
val_data = FB15k_237('../data/FB15k', split='val')[0]
test_data = FB15k_237('../data/FB15k', split='test')[0]

Processing...
Done!


In [ ]:
model = TransE(num_nodes=train_data.num_nodes,
                   num_relations=train_data.num_edge_types,
                   hidden_channels=50)

In [ ]:
loader = model.loader(head_index=train_data.edge_index[0],
                          rel_type=train_data.edge_type,
                          tail_index=train_data.edge_index[1],
                          batch_size=1000,
                          shuffle=True)


In [ ]:
optimizer = Adam(model.parameters(), lr=0.01)

In [ ]:
train(model, loader, optimizer)

Epoch: 000, Loss: 0.7610
Epoch: 001, Loss: 0.5589
Epoch: 003, Loss: 0.3506
Epoch: 004, Loss: 0.2960


In [ ]:
rank, mrr, hits10 = model.test(head_index=test_data.edge_index[0],
                                   rel_type=test_data.edge_type,
                                   tail_index=test_data.edge_index[1],
                                   batch_size=1000, k=10)

100%|██████████| 20466/20466 [22:08<00:00, 15.41it/s]


In [ ]:
hits1, hits3, hits10, mr, mrr = evaluate(model, loader)
print(f"Hits@1: {hits1}, Hits@3: {hits3}, Hits@10: {hits10}, MR: {mr}, MRR: {mrr}")

Hits@1: 0.003663003663003663, Hits@3: 0.003663003663003663, Hits@10: 0.007326007326007326, MR: 482.86446886446885, MRR: 0.008962223444461495


#Exercise 2 – Creating knowledge graph nesting networks with ComplEx

In [ ]:
model_2 = ComplEx(num_nodes=train_data.num_nodes,
                num_relations=train_data.num_edge_types,
                hidden_channels=50)

In [ ]:
loader = model_2.loader(head_index=train_data.edge_index[0],
                          rel_type=train_data.edge_type,
                          tail_index=train_data.edge_index[1],
                          batch_size=1000,
                          shuffle=True)

In [ ]:
optimizer = Adam(model_2.parameters(), lr=0.01)

In [ ]:
train(model_2, loader, optimizer)

Epoch: 000, Loss: 0.5509
Epoch: 001, Loss: 0.3061
Epoch: 002, Loss: 0.1768
Epoch: 003, Loss: 0.1322
Epoch: 004, Loss: 0.1135


In [ ]:
rank, mrr, hits10 = model_2.test(head_index=test_data.edge_index[0],
                                   rel_type=test_data.edge_type,
                                   tail_index=test_data.edge_index[1],
                                   batch_size=1000, k=10)

100%|██████████| 20466/20466 [04:29<00:00, 76.07it/s]


In [ ]:
hits1, hits3, hits10, mr, mrr = evaluate(model_2, loader)
print(f"Hits@1: {hits1}, Hits@3: {hits3}, Hits@10: {hits10}, MR: {mr}, MRR: {mrr}")

Hits@1: 0.0, Hits@3: 0.0, Hits@10: 0.003663003663003663, MR: 477.8937728937729, MRR: 0.005162883963139093


###Conclusion: TransE outperforms ComplEx in Hits@1, Hits@3, Hits@10, and MRR, indicating better top-k ranking accuracy. While ComplEx has a slightly better Mean Rank (MR), it is not enough to surpass TransE's overall performance in accuracy metrics. Thus, TransE is the preferred model for higher ranking precision.

#4.2

In [8]:
!pip install git+https://github.com/pykeen/pykeen.git

  Cloning https://github.com/pykeen/pykeen.git to /tmp/pip-req-build-g3vy6om3
  Running command git clone --filter=blob:none --quiet https://github.com/pykeen/pykeen.git /tmp/pip-req-build-g3vy6om3
  Resolved https://github.com/pykeen/pykeen.git to commit 98917dfc2c67606e865cb20d836b3b877b1828f3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 21.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-m

In [9]:
from torch.optim import Adam
from pykeen import predict
from pykeen.models import TransE
from pykeen.datasets import FB15k237, Nations
from pykeen.training import SLCWATrainingLoop
from pykeen.evaluation import RankBasedEvaluator
import matplotlib.pyplot as plt
from pykeen.models import TransE
from pykeen.pipeline import pipeline
from pykeen.datasets import FB15k237, Nations
from pykeen.training import SLCWATrainingLoop
from pykeen.evaluation import RankBasedEvaluator
from pykeen.sampling import BasicNegativeSampler
from pykeen import predict

INFO:pykeen.utils:Using opt_einsum


In [ ]:
import warnings

In [ ]:
warnings.filterwarnings("ignore")

#Task 3 – Creating network nesting for a knowledge graph with TransE with the PyKEEN library

In [ ]:
dataset = FB15k237()

In [ ]:
print(dataset.training.entity_id_to_label)
print(dataset.training.relation_id_to_label)
print(dataset.training.mapped_triples)

INFO:pykeen.datasets.base:downloading data from https://download.microsoft.com/download/8/7/0/8700516A-AB3D-4850-B4BB-805C515AECE1/FB15K-237.2.zip to /root/.data/pykeen/datasets/fb15k237/FB15K-237.2.zip


{0: '/m/010016', 1: '/m/0100mt', 2: '/m/0102t4', 3: '/m/0104lr', 4: '/m/0105y2', 5: '/m/0106dv', 6: '/m/0108xl', 7: '/m/0109vk', 8: '/m/010bnr', 9: '/m/010bxh', 10: '/m/010cw1', 11: '/m/010dft', 12: '/m/010h9y', 13: '/m/010hn', 14: '/m/010m55', 15: '/m/010nlt', 16: '/m/010p3', 17: '/m/010r6f', 18: '/m/010rvx', 19: '/m/010t4v', 20: '/m/010tkc', 21: '/m/010v8k', 22: '/m/010xjr', 23: '/m/010y34', 24: '/m/010z5n', 25: '/m/0113sg', 26: '/m/0114m0', 27: '/m/0118d3', 28: '/m/011_3s', 29: '/m/011_6p', 30: '/m/011_vz', 31: '/m/011hdn', 32: '/m/011hq1', 33: '/m/011j5x', 34: '/m/011k11', 35: '/m/011k1h', 36: '/m/011k4g', 37: '/m/011k_j', 38: '/m/011kn2', 39: '/m/011lpr', 40: '/m/011lvx', 41: '/m/011pcj', 42: '/m/011s0', 43: '/m/011s9r', 44: '/m/011v3', 45: '/m/011vx3', 46: '/m/011w20', 47: '/m/011w4n', 48: '/m/011w54', 49: '/m/011wdm', 50: '/m/011wtv', 51: '/m/011x_4', 52: '/m/011xg5', 53: '/m/011xhx', 54: '/m/011xjd', 55: '/m/011xy1', 56: '/m/011ycb', 57: '/m/011yd2', 58: '/m/011ydl', 59: '/m/01

In [ ]:
sample_triple = dataset.training.mapped_triples[0].detach().cpu().numpy()

In [ ]:
triple = [dataset.training.entity_id_to_label[sample_triple[0]],
          dataset.training.relation_id_to_label[sample_triple[1]],
          dataset.training.entity_id_to_label[sample_triple[2]]]

In [ ]:
print(sample_triple)
print(triple)

[    0   120 13647]
['/m/010016', '/location/capital_of_administrative_division/capital_of./location/administrative_division_capital_relationship/administrative_division', '/m/0mr_8']


In [ ]:
model = TransE(triples_factory=dataset.training)
optimizer = Adam(params=model.get_grad_params())
trainer = SLCWATrainingLoop(model=model,
                            triples_factory=dataset.training,
                            optimizer=optimizer)

In [ ]:
trainer.train(triples_factory=dataset.training,
              num_epochs=20,
              batch_size=64)

Training epochs on cpu:   0%|          | 0/20 [00:00<?, ?epoch/s]

Training batches on cpu:   0%|          | 0/4252 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/4252 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/4252 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/4252 [00:00<?, ?batch/s]

...

Training batches on cpu:   0%|          | 0/4252 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/4252 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/4252 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/4252 [00:00<?, ?batch/s]

[0.7498613756293692,
 0.28609470029730394,
 0.13319936449712216,
 0.09358500619703661,
 0.07824581296919991,
 0.07042401530346215,
 0.0661209862616162,
 0.063045999324361,
 0.06044198634314963,
 0.05762979040721384,
 0.056841319438283834,
 0.05432572918711887,
 0.05417082818209256,
 0.05238920519577895,
 0.05155707394961804,
 0.05061873253645167,
 0.04951056367730825,
 0.04926227272539868,
 0.04917227785231814,
 0.04857544448868015]

In [ ]:
evaluator = RankBasedEvaluator()

In [ ]:
subset_test_triples = dataset.testing.mapped_triples[:2000]

In [ ]:
res = evaluator.evaluate(model=model,
                         mapped_triples=subset_test_triples,
                         batch_size=128,
                         additional_filter_triples=[dataset.training.mapped_triples,
                                                    dataset.validation.mapped_triples])

Evaluating on cpu:   0%|          | 0.00/2.00k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 92.06s seconds


In [ ]:
print(f"Hits@1: {res.get_metric('hits_at_1')}\n"
      f"Hits@3: {res.get_metric('hits_at_3')}\n"
      f"Hits@10: {res.get_metric('hits_at_10')}\n"
      f"MR: {res.get_metric('mean_rank')}\n"
      f"MRR: {res.get_metric('mrr')}")

Hits@1: 0.08975
Hits@3: 0.16325
Hits@10: 0.2725
MR: 431.4986267089844
MRR: 0.15179720520973206


#Task 4 - Evaluation of network nestings for a graph of familiarity with the PyKEEN library

In [10]:
dataset = Nations()

In [11]:
dataset.training.entity_id_to_label

{0: 'brazil',
 1: 'burma',
 2: 'china',
 3: 'cuba',
 4: 'egypt',
 5: 'india',
 6: 'indonesia',
 7: 'israel',
 8: 'jordan',
 9: 'netherlands',
 10: 'poland',
 11: 'uk',
 12: 'usa',
 13: 'ussr'}

In [12]:
dataset.training.relation_id_to_label

{0: 'accusation',
 1: 'aidenemy',
 2: 'attackembassy',
 3: 'blockpositionindex',
 4: 'booktranslations',
 5: 'boycottembargo',
 6: 'commonbloc0',
 7: 'commonbloc1',
 8: 'commonbloc2',
 9: 'conferences',
 10: 'dependent',
 11: 'duration',
 12: 'economicaid',
 13: 'eemigrants',
 14: 'embassy',
 15: 'emigrants3',
 16: 'expeldiplomats',
 17: 'exportbooks',
 18: 'exports3',
 19: 'independence',
 20: 'intergovorgs',
 21: 'intergovorgs3',
 22: 'lostterritory',
 23: 'militaryactions',
 24: 'militaryalliance',
 25: 'negativebehavior',
 26: 'negativecomm',
 27: 'ngo',
 28: 'ngoorgs3',
 29: 'nonviolentbehavior',
 30: 'officialvisits',
 31: 'pprotests',
 32: 'relbooktranslations',
 33: 'reldiplomacy',
 34: 'releconomicaid',
 35: 'relemigrants',
 36: 'relexportbooks',
 37: 'relexports',
 38: 'relintergovorgs',
 39: 'relngo',
 40: 'relstudents',
 41: 'reltourism',
 42: 'reltreaties',
 43: 'severdiplomatic',
 44: 'students',
 45: 'timesinceally',
 46: 'timesincewar',
 47: 'tourism',
 48: 'tourism3',


In [13]:
dataset.training.mapped_triples

tensor([[ 0,  3,  2],
        [ 0,  3,  3],
        [ 0,  3, 10],
        ...,
        [13, 54,  9],
        [13, 54, 11],
        [13, 54, 12]])

In [14]:
model = TransE(triples_factory=dataset.training)
optimizer = Adam(params=model.get_grad_params())
trainer = SLCWATrainingLoop(model=model,
                            triples_factory=dataset.training,
                            optimizer=optimizer)

In [15]:
trainer.train(triples_factory=dataset.training,
                  num_epochs=10,
                  batch_size=64)

Training epochs on cpu:   0%|          | 0/10 [00:00<?, ?epoch/s]

Training batches on cpu:   0%|          | 0/25 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/25 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/25 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/25 [00:00<?, ?batch/s]

...

Training batches on cpu:   0%|          | 0/25 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/25 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/25 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/25 [00:00<?, ?batch/s]

[1.3584366893768312,
 1.3791724801063538,
 1.3426819801330567,
 1.2515608835220338,
 1.2512185502052307,
 1.2123078989982605,
 1.1394022083282471,
 1.1306866383552552,
 1.1423381185531616,
 1.0916993427276611]

In [16]:
evaluator = RankBasedEvaluator()

In [17]:
res = evaluator.evaluate(model=model,
                             mapped_triples=dataset.testing.mapped_triples,
                             batch_size=128,
                             additional_filter_triples=[dataset.training.mapped_triples,
                                                        dataset.validation.mapped_triples])

Evaluating on cpu:   0%|          | 0.00/201 [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.38s seconds


In [18]:
uk_predictions = predict.predict_target(model=model,
                                 head="uk",
                                 relation="intergovorgs",
                                 triples_factory=dataset.training).filter_triples(dataset.testing).df

In [19]:
usa_predictions = predict.predict_target(model=model,
                                  head="usa",
                                  relation="intergovorgs",
                                  triples_factory=dataset.training).filter_triples(dataset.testing).df

In [20]:
print("Predictions for UK:")
print(uk_predictions)

Predictions for UK:
    tail_id      score   tail_label
11       11  -6.861503           uk
5         5  -9.204095        india
7         7  -9.505027       israel
12       12  -9.644714          usa
13       13  -9.991801         ussr
4         4 -10.107436        egypt
10       10 -10.206842       poland
3         3 -10.607315         cuba
2         2 -10.703760        china
6         6 -10.703969    indonesia
9         9 -10.750030  netherlands
8         8 -11.056730       jordan
0         0 -11.255136       brazil
1         1 -11.351039        burma


In [21]:
print("Predictions for USA:")
print(usa_predictions)

Predictions for USA:
    tail_id      score   tail_label
12       12  -6.861503          usa
5         5  -8.603150        india
7         7  -9.026627       israel
6         6  -9.149668    indonesia
2         2  -9.253640        china
8         8  -9.345556       jordan
3         3  -9.373374         cuba
4         4  -9.423335        egypt
10       10  -9.588258       poland
9         9  -9.759627  netherlands
11       11  -9.846982           uk
13       13  -9.883624         ussr
0         0 -10.724046       brazil
1         1 -11.219618        burma


In [22]:
countries = dataset.testing.entity_to_id
relations = dataset.testing.relation_to_id

In [23]:
def get_actual_relations_for_country(country):
    country_id = countries[country]
    country_relations = []
    for h, r, t in dataset.testing.mapped_triples:
        if h == country_id or t == country_id:
            head = list(countries.keys())[list(countries.values()).index(h)]
            tail = list(countries.keys())[list(countries.values()).index(t)]
            relation = list(relations.keys())[list(relations.values()).index(r)]
            country_relations.append((head, relation, tail))
    return country_relations

In [24]:
def get_predicted_relations_for_country(country):
    predicted_relations_df = predict.predict_target(
        model=model,
        head=country,
        relation="intergovorgs",
        triples_factory=dataset.training
    ).filter_triples(dataset.testing).df
    predicted_relations = []
    for _, row in predicted_relations_df.iterrows():
        head = country
        tail_id = row['tail_id']
        tail = list(countries.keys())[list(countries.values()).index(tail_id)]
        relation = 'intergovorgs'  # Since we are predicting for this relation
        predicted_relations.append((head, relation, tail))
    return predicted_relations

In [25]:
actual_relations_uk = get_actual_relations_for_country('uk')
predicted_relations_uk = get_predicted_relations_for_country('uk')

actual_relations_usa = get_actual_relations_for_country('usa')
predicted_relations_usa = get_predicted_relations_for_country('usa')

In [26]:
print("Comparison for UK:")
print("Actual Relations:")
print(actual_relations_uk)
print("Predicted Relations:")
print(predicted_relations_uk)

Comparison for UK:
Actual Relations:
[('brazil', 'embassy', 'uk'), ('burma', 'dependent', 'uk'), ('china', 'accusation', 'uk'), ('cuba', 'intergovorgs3', 'uk'), ('cuba', 'negativebehavior', 'uk'), ('cuba', 'reltreaties', 'uk'), ('egypt', 'commonbloc1', 'uk'), ('egypt', 'embassy', 'uk'), ('india', 'relexports', 'uk'), ('indonesia', 'accusation', 'uk'), ('indonesia', 'militaryactions', 'uk'), ('indonesia', 'negativecomm', 'uk'), ('indonesia', 'reldiplomacy', 'uk'), ('indonesia', 'violentactions', 'uk'), ('jordan', 'weightedunvote', 'uk'), ('netherlands', 'commonbloc2', 'uk'), ('netherlands', 'relexportbooks', 'uk'), ('netherlands', 'relintergovorgs', 'uk'), ('netherlands', 'tourism3', 'uk'), ('poland', 'independence', 'uk'), ('poland', 'weightedunvote', 'uk'), ('uk', 'conferences', 'indonesia'), ('uk', 'duration', 'indonesia'), ('uk', 'embassy', 'netherlands'), ('uk', 'intergovorgs3', 'brazil'), ('uk', 'intergovorgs3', 'netherlands'), ('uk', 'ngo', 'poland'), ('uk', 'pprotests', 'ussr'),

In [27]:
print("\nComparison for USA:")
print("Actual Relations:")
print(actual_relations_usa)
print("Predicted Relations:")
print(predicted_relations_usa)


Comparison for USA:
Actual Relations:
[('brazil', 'exports3', 'usa'), ('brazil', 'intergovorgs3', 'usa'), ('burma', 'blockpositionindex', 'usa'), ('burma', 'relintergovorgs', 'usa'), ('china', 'intergovorgs3', 'usa'), ('cuba', 'independence', 'usa'), ('cuba', 'timesincewar', 'usa'), ('egypt', 'independence', 'usa'), ('egypt', 'ngoorgs3', 'usa'), ('egypt', 'reldiplomacy', 'usa'), ('india', 'relexports', 'usa'), ('india', 'reltreaties', 'usa'), ('india', 'students', 'usa'), ('indonesia', 'militaryalliance', 'usa'), ('indonesia', 'relngo', 'usa'), ('indonesia', 'students', 'usa'), ('israel', 'intergovorgs3', 'usa'), ('israel', 'relngo', 'usa'), ('jordan', 'ngoorgs3', 'usa'), ('jordan', 'reldiplomacy', 'usa'), ('jordan', 'relngo', 'usa'), ('netherlands', 'treaties', 'usa'), ('poland', 'reldiplomacy', 'usa'), ('poland', 'relexports', 'usa'), ('poland', 'relintergovorgs', 'usa'), ('uk', 'relngo', 'usa'), ('uk', 'timesinceally', 'usa'), ('usa', 'accusation', 'cuba'), ('usa', 'blockpositionin

###Do the predictions match the actual data?

In [31]:
def compare_relations(actual_relations, predicted_relations):
    actual_set = set(actual_relations)
    predicted_set = set(predicted_relations)

    matching = actual_set.intersection(predicted_set)
    only_actual = actual_set.difference(predicted_set)
    only_predicted = predicted_set.difference(actual_set)

    return matching, only_actual, only_predicted

def check_matching(matching, only_actual, only_predicted):
    if not only_actual and not only_predicted:
        return "Yes"
    else:
        return "No"

In [32]:
matching_uk, only_actual_uk, only_predicted_uk = compare_relations(actual_relations_uk, predicted_relations_uk)
uk_match_result = check_matching(matching_uk, only_actual_uk, only_predicted_uk)
print(f"Do the predictions for UK match the actual data? {uk_match_result}")

matching_usa, only_actual_usa, only_predicted_usa = compare_relations(actual_relations_usa, predicted_relations_usa)
usa_match_result = check_matching(matching_usa, only_actual_usa, only_predicted_usa)
print(f"Do the predictions for USA match the actual data? {usa_match_result}")

Do the predictions for UK match the actual data? No
Do the predictions for USA match the actual data? No


######Not all of the predictions match the actual data